## Tensorflow VS Keras

Tensorflow is a low-level library like NumPy. Keras is a high-level API built on top of a low-level library or framework. Nowadays mostly done on Tensorflow.

Tensorflow is a rich low-level ecosystem and Keras brings an easy-to-use layer of abstraction to it.

### Load Dataset

From Tensorflow directly. Later also from CSV.

In [117]:
import tensorflow as tf

In [118]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [119]:
X_train, X_test = X_train / 255.0, X_test / 255.0

### Build Model / Architecture

In [122]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

model = Sequential()
model.add(Input((28,28)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10))  # no softmax here directly (for numerically stable loss calculation)

In [123]:
nonsense_prediction = model(X_train[:1]).numpy()
nonsense_prediction

array([[ 0.98965836, -0.16535057,  0.3153936 ,  0.7491257 , -0.37444824,
        -0.25511223, -1.1548389 , -0.66470635,  0.66366136,  0.58196604]],
      dtype=float32)

In [124]:
tf.nn.softmax(nonsense_prediction).numpy()  # more interpretable than the logit

array([[0.20619653, 0.06496319, 0.10506351, 0.16211358, 0.05270568,
        0.05938604, 0.02415116, 0.03942756, 0.14883418, 0.13715853]],
      dtype=float32)

In [125]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

loss_fn = SparseCategoricalCrossentropy(from_logits=True)

expected_loss = -tf.math.log(1/10)  # neg log likelihood for roughly 1/10 probability for each class

print('Expected loss on random guess', expected_loss)
print('Actual loss:', loss_fn(y_train[:1], nonsense_prediction))

Expected loss on random guess tf.Tensor(2.3025851, shape=(), dtype=float32)
Actual loss: tf.Tensor(2.8236961, shape=(), dtype=float32)


### Compile & Train Model

In [126]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.01)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [127]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 822us/step - accuracy: 0.9042 - loss: 0.3205
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 830us/step - accuracy: 0.9297 - loss: 0.2499
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 816us/step - accuracy: 0.9359 - loss: 0.2320
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 821us/step - accuracy: 0.9415 - loss: 0.2142
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 845us/step - accuracy: 0.9449 - loss: 0.2064
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 816us/step - accuracy: 0.9467 - loss: 0.2040
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 792us/step - accuracy: 0.9490 - loss: 0.1963
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 808us/step - accuracy: 0.9505 - loss: 0.1945
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 814us/step - accuracy: 0.9524 - loss: 0.1870
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 814us/step - accuracy: 0.9530 - loss: 0.1846


### Evaluate Model

In [128]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9646 - loss: 0.2085


[0.2084924280643463, 0.9646000266075134]

In [129]:
tf.argmax(tf.nn.softmax(model(X_test[:1])), axis=1)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([7])>

In [130]:
y_test[:1]

array([7], dtype=uint8)

### Regression Task

In [131]:
import pandas as pd

df = pd.read_csv('housing.csv')

In [132]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [133]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X, y = df.drop('MedHouseVal', axis=1), df['MedHouseVal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [134]:
X_train_scaled

array([[-0.35939723, -0.36901309, -0.17833968, ..., -0.11939012,
         0.96511061, -0.70750104],
       [-0.38734824, -1.16375719, -0.19667756, ..., -0.21984503,
         0.79149871, -1.12251693],
       [ 0.63428751,  1.22047512, -0.09570399, ..., -0.22315484,
        -0.69124073,  0.58254834],
       ...,
       [ 0.59452325, -0.84585955,  0.10348793, ..., -0.06137814,
         0.90880405, -1.21252037],
       [-0.85845459,  0.10783338,  0.87756234, ...,  0.20386278,
        -0.70062515,  1.11756882],
       [-0.1676955 ,  0.58467984,  0.0781359 , ..., -0.0939072 ,
         0.97449503, -1.29252343]], shape=(16512, 8))

In [135]:
X_train_scaled.shape

(16512, 8)

In [136]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(8,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

/home/neuralnine/Documents/Programming/NeuralNine/youtube-preparation/Tensorflow Crash Course/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [153]:
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'r2_score'])

In [154]:
model.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2877 - mae: 0.3703 - r2_score: 0.7851  
Epoch 2/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - loss: 0.2821 - mae: 0.3657 - r2_score: 0.7893
Epoch 3/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - loss: 0.2831 - mae: 0.3653 - r2_score: 0.7886
Epoch 4/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - loss: 0.2745 - mae: 0.3609 - r2_score: 0.7950
Epoch 5/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step - loss: 0.2751 - mae: 0.3604 - r2_score: 0.7946
Epoch 6/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: 0.2706 - mae: 0.3577 - r2_score: 0.7979
Epoch 7/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - loss: 0.2689 - mae: 0.3565 - r2_score: 0.7992
Epoch 8/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - loss: 0.2710 - mae: 0.3569 - r2_score: 0.7976
Epoch 9/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - loss: 0.2681 - mae: 0.3543 - r2_score: 0.7998
Epoch 10/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - loss: 0.2637 - mae: 0.352

In [155]:
model.evaluate(X_test_scaled, y_test)

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - loss: 0.2841 - mae: 0.3605 - r2_score: 0.7816


[0.2841041088104248, 0.3605435788631439, 0.7816312313079834]

In [156]:
model(X_test_scaled[:1])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.7576113]], dtype=float32)>

In [157]:
y_test[:1]

19746    0.637
Name: MedHouseVal, dtype: float64

In [159]:
df.MedHouseVal.describe()

count    20640.000000
mean         2.068558
std          1.153956
min          0.149990
25%          1.196000
50%          1.797000
75%          2.647250
max          5.000010
Name: MedHouseVal, dtype: float64